## SDP application for BB84 like protocol

We must first call the appropriate functions to implement the problem.

We have also defined an "operator" class to help us manipulate the fixtures in the big matrix T

The "reduce", "arrange" and "ReduceList" functions are use to simplifies the expression of the multiplication of multiple operators

The "NPAhierarchy" function builds the entire hierarchy at the level considered



In [1]:
import numpy as np
import picos
import cvxpy as cp
from scipy.stats import norm
import math
import sys
from pprint import pprint
from matplotlib import pyplot as plt

In [2]:
class operator:
    
    def __init__(self,h,m,o):  
        
        self.m = m  # measurement basis  
        self.o = o  # measurement projector, here we always consider outcome 0
        self.h = h  # Hilbert space  0 for Bob 1 for Eve
        
        
    def __repr__(self):
        rep = []
        i =0
        while i<(len(self.h)):
            if self.h[i]==0:
                rep = rep + ['B_({}|{})'.format(self.m[i], self.o[i])]
            elif self.h[i]==1:
                rep = rep + ['E_({}|{})'.format(self.m[i], self.o[i])]
            elif self.h[i]==-1:
                rep = rep + ['Id']
            i = i+1
            
        return ''.join(rep)
    
    
    def __hash__(self):
        return hash((str(self.h), str(self.m), str(self.o))) 
    
                    
    def __mul__(self, other):
        return operator(self.h+other.h, self.m+other.m, self.o+other.o)
    
    
    __rmul__=__mul__
              
        
def reduce(OP):  # Simplifies the expression of the multiplication of multiple operators


    i=0
    l = len(OP.m)-1
    while i < l:
        ah = OP.h[i]
        ao = OP.o[i]
        am = OP.m[i]
        bh = OP.h[i+1]
        bo = OP.o[i+1]
        bm = OP.m[i+1]
        if ah != bh:
            if ah==1:  # Place the operator of Eve far right and those of Bob far left 
                OP.h[i], OP.h[i+1] = OP.h[i+1], OP.h[i]
                OP.o[i], OP.o[i+1] = OP.o[i+1], OP.o[i]
                OP.m[i], OP.m[i+1] = OP.m[i+1], OP.m[i]
            elif ah==-1:  # If the identity operator is found, we simplify the expresson and remove it
                (OP.h).pop(i)
                (OP.o).pop(i)
                (OP.m).pop(i)
                l = l-1
                i = i-1
            elif bh==-1: # If the identity operator is found, we simplify the expresson and remove it
                (OP.h).pop(i+1)
                (OP.o).pop(i+1)
                (OP.m).pop(i+1)
                l = l-1
                i = i-1
                
            else:
                pass
        elif ah==bh:
            if ah==1:  # 1 correspond to Eve
                (OP.h).pop(i+1)
                (OP.o).pop(i+1)
                (OP.m).pop(i+1)
                l = l-1
                i = i-1
            elif ah==0:  # 0 correspond to Bob
                if am != bm:
                    pass
                else:
                    if ao==bo:
                        (OP.h).pop(i+1)
                        (OP.o).pop(i+1)
                        (OP.m).pop(i+1)
                        l = l-1
                        i = i-1   
                    else:
                        return 0
            elif ah==-1:
                (OP.h).pop(i+1)
                (OP.o).pop(i+1)
                (OP.m).pop(i+1)
                l = l-1
                i = i-1
                
  
        i = i+1
    return OP
        
        
def arrange(OPToReduce):
    if len(OPToReduce.m)==1:
        pass
    else:
        for i in range(len(OPToReduce.m)):
            reduce(OPToReduce)
    return reduce(OPToReduce)



def Transp(OP):
    if OP == 0:
        return 0
    else:
        h = OP.h
        o = OP.o
        m = OP.m
        T = operator(h[::-1],m[::-1],o[::-1])
        Transp = arrange(T)
        return T
    

def ReduceList(Mon):
    for i,m in enumerate(Mon):
        for j,n in enumerate(Mon):
            if i!=j:
                if hash(m)==hash(n):
                    Mon.pop(j)
                else:
                    pass
            else:
                pass
        
    return Mon



In [3]:
# mesure x outcome a , mesure 
def NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,lvl):
    Max = NOutcomeB*NMesureB+NOutcomeE*NMesureE +1
    
    if lvl == 1:
        Mon = [operator([-1],[0],[0])]
        MonT = [operator([-1],[0],[0])]
        for k in range(NMesureB):
            for i in range(NOutcomeB):
                Mon = Mon + [operator([0],[k],[i])]
                MonT = MonT + [operator([0],[k],[i])]
                
        for k in range(NMesureE):
            for i in range(NOutcomeE):
                Mon = Mon + [operator([1],[k],[i])]
                MonT = MonT + [operator([1],[k],[i])]
            
    elif lvl == 1.5:
        Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,1)
        for e in range(NMesureE):
            for eo in range(NOutcomeE):
                for k in range(NMesureB):
                    for i in range(NOutcomeB):
                        Mon = Mon + [(operator([0],[k],[i])*operator([1],[e],[eo]))]
                        MonT = MonT + [(operator([0],[k],[i])*operator([1],[e],[eo]))]
        
    elif lvl == 2:
        Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,1)
        for i,m in enumerate(Mon[1:Max]):
            for j,n in enumerate(Mon[1:Max]):
                #if i!=j:
                tot = arrange(n*m)
                totT = arrange(m*n)
                if tot !=0:
                    Mon = Mon + [tot]
                    MonT = MonT + [totT]
                else:
                    pass
                #else:
                 #   pass
    
    elif lvl == 3:
        Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,2)
        for i,m in enumerate(Mon[1:Max]):
            for j,n in enumerate(Mon[1:Max]):
                for k,p in enumerate(Mon[1:Max]):
                    #if i!=j and i!=k and k!=j:
                    tot = arrange(arrange(m*n)*p)
                    totT = arrange(arrange(p*n)*m)
                    if tot != 0:
                        Mon = Mon + [tot]
                        MonT = MonT + [totT]
                    else:
                        pass
                    #else:
                        #pass
        
        
    else:
        pass
    
    Mon = ReduceList(Mon)
    MonT = ReduceList(MonT)
    return Mon, MonT

Once those function are defined, we can construct a dictionary of the different element in the big matrix T.

This allows us to reduce the number of variables for the problem.

In [5]:
NMesureB = 2
NOutcomeB = 1
NMesureE = 1
NOutcomeE = 1
LVLNPA = 2
states = 4
print('NPA LVL{}'.format(LVLNPA))
Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,LVLNPA)

#Mon = ReduceList(Mon)
#MonT = ReduceList(MonT)
            
        

MAX = len(Mon)
print('Sub-block: {}x{}'.format(MAX,MAX))
print('Big matrix: {}x{}'.format(states*MAX,states*MAX))
print(Mon)
print(MonT)
terms = {}
#termsT = {}

for i,m in enumerate(Mon):
    for j,n in enumerate(MonT):
        tot = arrange(n*m)
        tot_rep = hash(tot)
        if tot_rep in terms:
            terms[tot_rep].append((j,i))
        else:
            terms[tot_rep] = [(j,i)]

print("Number of different elements in a sub-block :", len(terms.keys()))
ndif = len(terms.keys())*((states*(states+1))//2)
print("Number of different variables in the problem :",ndif)

NPA LVL2
Sub-block: 8x8
Big matrix: 32x32
[Id, B_(0|0), B_(1|0), E_(0|0), B_(1|0)B_(0|0), B_(0|0)E_(0|0), B_(0|0)B_(1|0), B_(1|0)E_(0|0)]
[Id, B_(0|0), B_(1|0), E_(0|0), B_(0|0)B_(1|0), B_(0|0)E_(0|0), B_(1|0)B_(0|0), B_(1|0)E_(0|0)]
Number of different elements in a sub-block : 16
Number of different variables in the problem : 160


Once the separate elements have been identified, the big matrix T is built, making sure that it is Hermitian

We implement then the Gram constraints and the statistical constraints

# CVXPY

In [ ]:

Matlist = []
for X in range(4):
    for Y in range(X,4):
        for k in terms.keys():
            B = np.zeros((4*MAX,4*MAX))
            if k ==0:
                Matlist.append(B)
            else:
                for i in terms[k]:
                    B[X*MAX+i[0],Y*MAX+i[1]] = 1    
                Matlist.append(B)

if len(Matlist) != ndif:
    print("Mismatch parameter number, shoud be :",ndif, " insted of :",len(Matlist))
    print("end program")
    sys.exit(1)
    
print("Number parameter match")
print("Summing matrix")

z = cp.Variable(ndif,complex=True)
T = sum([(Matlist[i]*z[i] + Matlist[i].T*cp.conj(z[i])) for i in range(ndif)])

    
GguessBlC = []
GguessAlC = []
GHBElc = []
GHAElc = []
GHBAlc = []
GKeyRc = []
GKeyDc = []
GARange = [x*0.1 for x in range(10*0, 20*1)][1:]
for a in GARange:
    print('a:',a)
    constraints = [T >> 0]
    guessBlC = []
    guessAlC = []
    HBElc = []
    HAElc = []
    HBAlc = []
    KeyRc = []
    KeyDc = []
    
#Gram Matrix
    Gram = np.array([[1,np.exp(-2*a**2),np.exp(-a**2*(1+1j)),np.exp(-a**2*(1-1j))],
                   [np.exp(-2*a**2),1,np.exp(-a**2*(1-1j)),np.exp(-a**2*(1+1j))],
                   [np.exp(-a**2*(1-1j)),np.exp(-a**2*(1+1j)),1,np.exp(-2*a**2)],
                   [np.exp(-a**2*(1+1j)),np.exp(-a**2*(1-1j)),np.exp(-2*a**2),1]],dtype=complex)

    for X in range(4):
        for Y in range( 4):
            constraints.append(T[MAX*X, MAX*Y] == Gram[X, Y])

        
    interval = -math.floor(-a*2) + math.ceil(a*2)

    stepp = 2*interval/NOutcomeB
    B = int(math.floor(-a*2)/stepp)
    A = int(math.ceil(a*2)/stepp)
    print("A =",A)
    print("B =",B)
    Range = [[x*stepp,(x+1)*stepp] for x in range(B, A)]
    print('Range :',Range)

    positionE = 1+2*NOutcomeB
    mid = NOutcomeB/2
    k = 1
    if mid < 1.:
        PguesB = T[0,0]+T[MAX,MAX]+2*T[1,3]-T[1,1]-T[3,3]+2*T[MAX+1,MAX+3]-T[MAX+1,MAX+1]-T[MAX+3,MAX+3]     
    else:
        PguesB = 0
        while (k <= mid):
            PguesB = PguesB + T[k+mid,positionE] - T[k,positionE] + T[k,k] \
            + T[MAX+k+mid,MAX+positionE] - T[MAX+k,MAX+positionE] + T[MAX+k,MAX+k]
            k = k + 1 
            
    PguesA = (T[positionE,positionE] + T[MAX,MAX] - T[MAX+positionE,MAX+positionE])/2

    PGuessB = cp.real(PguesB/2)
    PGuessA = cp.real(PguesA)

    solution = []
    distanceC =   [x*30 for x in range(10*0, 4*1)]#[0,1,50,100,150,200]#[x*20 for x in range(10*0, 10*1)]
    for d in distanceC:
        Qstata = []
        Qstat_a =[]
        Qstatia = []
        Qstat_ia = []
        #d = 1
        print('d value =',d)
        att = 0.02
        n = pow(10,-att*d)   
        Pqa = norm(np.sqrt(2*n)*a, 1/np.sqrt(2))
        Pq_a = norm(-np.sqrt(2*n)*a, 1/np.sqrt(2))
        Pqia = norm(0, 1/np.sqrt(2))
        Pq_ia = norm(0, 1/np.sqrt(2))
    
        if NOutcomeB==1:
            #print('Range Statistique : [-inf,0],[0,inf]')
            Qstata = [1-Pqa.cdf(0)]
            Qstat_a =[1-Pq_a.cdf(0)]
            Qstatia = [1-Pqia.cdf(0)]
            Qstat_ia = [1-Pq_ia.cdf(0)]
        
        else:
            print('Range Statistique : [-inf,{}],{},[{}, inf]'.format(A,Range,B))
            Qstata = [Pqa.cdf(Range[0][0])]
            Qstat_a =[Pq_a.cdf(Range[0][0])]
            Qstatia = [Pqia.cdf(Range[0][0])]
            Qstat_ia = [Pq_ia.cdf(Range[0][0])]
        
            for i in range(len(Range)):
                aa = Range.copy()[i][0]
                b = Range.copy()[i][1]
                
                Qstata = Qstata+ [Pqa.cdf(b)-Pqa.cdf(aa)]
                Qstat_a = Qstat_a+ [Pq_a.cdf(b)-Pq_a.cdf(aa)]
                Qstatia = Qstatia+ [Pqia.cdf(b)-Pqia.cdf(aa)]
                Qstat_ia = Qstat_ia+ [Pq_ia.cdf(b)-Pq_ia.cdf(aa)]

            Qstata = Qstata+ [1-Pqa.cdf(Range[-1][-1])]
            Qstat_a = Qstat_a+ [1-Pq_a.cdf(Range[-1][-1])]
            Qstatia = Qstatia+ [1-Pqia.cdf(Range[-1][-1])]
            Qstat_ia = Qstat_ia+ [1-Pq_ia.cdf(Range[-1][-1])]
        print('Qstata :', Qstata)
        print('Qstat_a :', Qstat_a)
        print('Qstatia :', Qstatia)
        print('Qstat_ia :', Qstat_ia)
        countq = 1
        countp = 1+NOutcomeB
        constraints2 = []
        nameq = ['|α>','|-α>','|iα>','|-iα>']
    
        for j in (Qstata,Qstat_a,Qstatia,Qstat_ia):  
            for i,c in enumerate(j):
                #print("i :",i)
                #print("c :",c)
                constraints2.append(T[countq+i,countq+i] == c)
            countq = countq + MAX 
        for j in (Qstatia,Qstat_ia,Qstata,Qstat_a):  
            for i,c in enumerate(j):
                constraints2.append(T[countp+i,countp+i] == c)
            countp = countp + MAX
        
        probB = cp.Problem(cp.Maximize(PGuessB),
                          constraints+constraints2)

        probA = cp.Problem(cp.Maximize(PGuessA),
                               constraints+constraints2) 

        probB.solve(solver=cp.MOSEK)
        probA.solve(solver=cp.MOSEK)
        #bound = fx(val)
        guessB = probB.value
        guessA = probA.value
        print("optimal value with MOSEK guessB =", guessB, "status", probB.status)
        print("optimal value with MOSEK guessA =", guessA, "status", probA.status)
    
        #fxl.append(bound)
        guessBlC.append(guessB)
        guessAlC.append(guessA)
    
        HBA = 0
        if NOutcomeB > 1:
            for i,c in enumerate(Qstata):
                HBA = HBA - c*math.log2(c)/2 
        else:
            HBA = HBA - Qstata[0]*math.log2(Qstata[0])/2 - (1-Qstata[0])*math.log2((1-Qstata[0]))/2
        
        #if guessB < 0:
         #   InfHPguessBc = 1000
        #elif 0<guessB:
        InfHPguessBc = -math.log2(guessB)
        #if guessA < 0:
         #   InfHPguessAc = 1000
        #elif 0<guessA:
        InfHPguessAc = -math.log2(guessA)
        
        HBElc.append(InfHPguessBc)
        HBAlc.append(InfHPguessAc)
        HBAlc.append(HBA)
    
        #print("H(B|A)=" ,HBA)
        #print("H(B|E)>=",InfHPguessBh)
        KeyRateReversc = InfHPguessBc - HBA
        KeyRateDirectc = InfHPguessAc - HBA
        if KeyRateReversc < 0:
            KeyRc.append(0)
        elif KeyRateReversc > 0:
            KeyRc.append(KeyRateReversc)
        if KeyRateDirectc < 0:
            KeyDc.append(0)
        elif KeyRateDirectc > 0:
            KeyDc.append(KeyRateDirectc)
    
        
    
        print("*****************")
    GguessBlC.append([a,guessBlC])
    GguessAlC.append([a,guessAlC])
    GHBElc.append([a,HBElc])
    GHAElc.append([a,HAElc])
    GHBAlc.append([a,HBAlc])
    GKeyRc.append([a,KeyRc])
    GKeyDc.append([a,KeyDc])

    print('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
    
    
print("DONE")